#**HOMEWORK 1: BLUE BIKE TRIP DURATION PREDICTION (Total: / 25 points)**

Context: You have received some data from Blue Bikes (the Boston Bikesharing Service). They have asked you to provide a predictive model that can accurately predict how long a particular bike trip will take, from one station to another. The use case is that a customer will open their bike sharing app, and have the option to select a starting and ending station, and receive back a predicted trip duration (in seconds). 

# *Names: (Student One, Student Two)*

Make sure that you answer the four questions provided at the front of the notebook (edit the XXX response sections to provide your answers). You probably want to answer these last, after you finish the hands-on portion of the assignment. 

#**Answers to Written Questions (10 points)**

**Question 1: What features did you opt to keep, and which did you discard? Why? (2.5 points)**

The names of the stations are probably not useful, given you have IDs and also latitudes and longitudes. If you keep these and parsed the text, etc., that was probably overkill and unnecessarily complicates the model. 

Most imporatantly, however: you should not have used the "stop time" in your model! If you did so, that's an information leakage problem. Most trips last less than one hour. So, if you give your model both the start and stop minute, it has a very easy time figuring out that it can take the second value minus the first (or 60+ the second minus the first) to get a reliable predictor of trip duration. Also, just think about it... if you are in the real world generate one of these predictions, you won't know the stop time at the time you are producing a prediction... 

If you used the lat and longitude of the ending station, that's okay in my book. It really depends on the use case for the predictions. Will the rider be able to tell you / your system where they are trying to go? If so, its fine for that predictor to enter the model. 

**Question 2: What transformations did you apply to the data, in terms of pre-procesing? Why? What feature engineering can you do here to help the model along? (2.5 points)**

Taking latitude and longitudes of start and stop stations as predictors, these are continuous numeric values. The same goes for the start time. You'd want to 'whiten' the input features before passing them to your model.

The most obvious feature engineering option here related to the geolocation data about the docking stations. You can calculate a trip distance variable, rather than have your model try to "figure it out." Haversine distance is a simple option (or even Euclidean distance). Perhaps you went to Google Maps API and pulled in other features of the location? Up to you. 

**Question 3: What activation functions did you consider for the output layer? Which did you rule out? Why? (2.5 points)**

This is a non-negative value that we are predicting. That means we need an activation function that can return non-negative integers. Linear (no) activation of course will work, but so would a ReLU. You could play with this. You definitely do not want to be using Sigmoid, TanH or Softmax here. 

**Question 4: What steps did you take to ensure the robustness of your model's performance, e.g., to avoid overfitting, or compatibility with new samples of data? (2.5 points)**

Standard idea here: cross-validation, train_test_split, etc. Any approach like this is fine in my view. As far as compatibility with new data, you want to make sure the one-hot encodings don't break the model. If I one-hot encode bike IDs, or station IDs, and use those, it's possible that a new bike or station ID will appear in later observations. Your model won't know how to process them! Think about it, if some bike IDs don't show up in the hold out sample, or if new ones do, when I pre-process the IDs to one-hot encode them, my resulting matrix will have a different number of columns than yours did. That means I have a different number of features. That means your model will throw an error when I try and pass the data for a prediction (or, you'll get lucky, the numbers will line up, but the prediction will be terrible, because the meaning of the features has changed). 

#*Import and Pre-process Data*

In [1]:
!pip install haversine

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import requests
import json
from haversine import haversine

bluebikes = pd.read_csv('https://raw.githubusercontent.com/gburtch/BA865-2022/main/Week%203/datasets/bluebikes_sample.csv')

# This function MUST return a pair of objects (predictors, labels, in that order) as numpy arrays.
def processData(data, makeDist='Haversine'):
    
    startmin = []
    for i in range(len(data)):
        startmin.append(int(data.loc[i,'starttime'].split(":")[0]))

    data['startmin'] = startmin
    usertype_bin = np.where(data['usertype']=='Subscriber',1,0)

    # This will throw away string variables.
    data = data.select_dtypes([np.number])
    
    # Here is how we could one-hot encode our data. 
    gender_onehot = utils.to_categorical(data['gender'])

    # You will hopefully have realized that you need to be very careful with the features you choose to keep in your model!!
    # It's quite possible that you will encounter a new bike ID in the holdout sample that did not appear in the training data.
    # The same can happen with station IDs (though it's less likely). 
    # If this happens, it will produce an error when your code is executed, unless it's handled very carefully. 
    # These identifiers do not seem to add a lot of value in prediction anyway, so I did not use them.  
    end_station_onehot = utils.to_categorical(data['end station id'])
    start_station_onehot = utils.to_categorical(data['start station id'])
    bike_onehot = utils.to_categorical(data['bikeid'])
    
    # If we are doing the distance construction, can either use Haversine, or try Open Street Maps.
    # I am doing some feature engineering here; use lat and lng to construct travel distances / durations. 
    if makeDist == 'Haversine':
        
        # First, let's get the list of unique pairs of start and end station coordinates. 
        # That way we don't need to duplicate effort.
        station_cols = data.loc[:,['start station latitude','start station longitude','end station latitude','end station longitude']]
        unique_pairs = station_cols.value_counts(ascending=True).reset_index(name='count').to_numpy()[:,0:4]
        
        # Put distances for unique pairs into a dictionary for lookups.
        # In theory we could do this for pairs in 'either' direction.
        dist = {}
        for i in range(len(unique_pairs)):
            dist[tuple(unique_pairs[i,:])] = haversine(station_cols.loc[i,['start station latitude','start station longitude']],
                                                       station_cols.loc[i,['end station latitude','end station longitude']])

        hav_dist = []
        for i in range(len(data)):
            index = data.loc[i,['start station latitude','start station longitude','end station latitude','end station longitude']]
            station_dist = dist[tuple(index)]
            hav_dist.append(station_dist)

        data['hav_dist'] = hav_dist
        predictors_cont = data[['start station latitude','start station longitude','end station latitude','end station longitude','birth year','startmin','hav_dist']].to_numpy()
        
    # Here we are doing trip duration queries using OpenStreetMaps
    elif makeDist == 'OSM':
        
        dist = []
        for i in range(len(data)):
    
            url = f"http://router.project-osrm.org/route/v1/car/{data.loc[i,['start station longitude']].to_numpy()[0]},{data.loc[i,['start station latitude']].to_numpy()[0]};{data.loc[i,['end station longitude']].to_numpy()[0]},{data.loc[i,['end station latitude']].to_numpy()[0]}?overview=false"
            # call the OSMR API
            r = requests.get(url)

            # then you load the response using the json libray
            # by default you get only one alternative so you access 0-th element of the `routes`
            routes = json.loads(r.content)
            shortest_duration = routes.get("routes")[0]['duration']
            dist.append(shortest_duration)
        
        data['travel_time'] = dist
        predictors_cont = data[['start station latitude','start station longitude','end station latitude','end station longitude','birth year','startmin','travel_time']].to_numpy()
    else:
        predictors_cont = data[['start station latitude','start station longitude','end station latitude','end station longitude','birth year','startmin']].to_numpy()

    # Pulling out continuous predictors, and 'normalizing' them.
    # You could also accomplish this with a BatchNormalization() layer in your model.
    predictors_cont = np.subtract(predictors_cont,np.mean(predictors_cont,axis=0).reshape(1,predictors_cont.shape[1]))
    predictors_cont = np.divide(predictors_cont,np.std(predictors_cont,axis=0).reshape(1,predictors_cont.shape[1]))

    # Putting everything back together.
    data = np.concatenate((data[['tripduration']].to_numpy(),predictors_cont,usertype_bin.reshape(len(usertype_bin),1),gender_onehot),axis=1) #bike_onehot,start_station_onehot,end_station_onehot
    
    # Create the labels vector and the matrix of predictors.
    labels = data[:,0]
    predictors = data[:,1:]
    
    train_labels = labels
    train_predictors = predictors

    return train_predictors, train_labels


#*Specify Your Neural Network Architecture, Process Your Sample*

Calling the data pre-processing function on the sample.

In [3]:
import warnings
warnings.filterwarnings('ignore')

predictors, labels = processData(bluebikes)

Specifying my Neural Network's structure. Note that the important thing for performance with this model actually comes down to its depth! It turns out that width isn't that important here. 


In [4]:
def build_model():
    model = keras.Sequential([
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mae", metrics=["mae"])
    return model

#*Train Your Neural Network Here*

In [5]:
folds = 2
num_val_samples = len(predictors) // folds # floor division (i.e., round down to nearest integer.)
num_epochs = 150
batch_sizes = 75
all_train_mae_histories, all_val_mae_histories = [],[]  

for i in range(folds): # the folds are going to be indexed 0 through 3 if k = 4
    print("Processing fold #:",i)
    
    val_data = predictors[i * num_val_samples: (i + 1) * num_val_samples] 
    val_targets = labels[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [predictors[:i * num_val_samples],
         predictors[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [labels[:i * num_val_samples],
         labels[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()

    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=batch_sizes)
    
    train_mae_history = history.history['mae']
    val_mae_history = history.history['val_mae']

    all_train_mae_histories.append(train_mae_history)
    all_val_mae_histories.append(val_mae_history)

average_train_mae_history = [np.mean([x[i] for x in all_train_mae_histories]) for i in range(num_epochs)]
average_val_mae_history = [np.mean([x[i] for x in all_val_mae_histories]) for i in range(num_epochs)]

Processing fold #: 0
Epoch 1/150
60/60 [==============================] - 3s 11ms/step - loss: 850.1215 - mae: 850.1215 - val_loss: 849.3384 - val_mae: 849.3384
Epoch 2/150
60/60 [==============================] - 0s 8ms/step - loss: 849.7796 - mae: 849.7796 - val_loss: 849.0845 - val_mae: 849.0845
Epoch 3/150
60/60 [==============================] - 0s 7ms/step - loss: 849.3654 - mae: 849.3654 - val_loss: 848.7019 - val_mae: 848.7019
Epoch 4/150
60/60 [==============================] - 0s 7ms/step - loss: 848.8765 - mae: 848.8765 - val_loss: 848.2066 - val_mae: 848.2066
Epoch 5/150
60/60 [==============================] - 0s 7ms/step - loss: 848.3134 - mae: 848.3134 - val_loss: 847.6241 - val_mae: 847.6241
Epoch 6/150
60/60 [==============================] - 0s 7ms/step - loss: 847.6766 - mae: 847.6766 - val_loss: 846.9727 - val_mae: 846.9727
Epoch 7/150
60/60 [==============================] - 0s 8ms/step - loss: 846.9662 - mae: 846.9662 - val_loss: 846.3223 - val_mae: 846.3223
Epoch

Plot your average training and validation MAE loss across epochs here:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(average_train_mae_history, c="r")
plt.plot(average_val_mae_history,c="b")
plt.legend(['Train','Validation'],title="Loss")
plt.xlabel("Epochs")
plt.ylabel("Average Validation MAE Across k Folds")
plt.show()

#*Choose Final Configuration and Produce That Model Here:*

In [ ]:
model = build_model()
model.fit(predictors,labels,epochs=80, batch_size=50)

Here's what the resulting model looks like.

In [ ]:
model.summary()

#*Final Evaluation*

Don't modify this section; this is the code I will use to evaluate that your model is output properly and that it can generate predictions on new test observations it has never seen before. If your model breaks when I feed it the new data, I will deduct marks, so please ensure that your data pre-processing function works properly!

In [ ]:
from google.colab import files
import io

# I'm going to upload my holdout dataset (same set of features)
uploaded = files.upload()
bluebike_holdout = pd.read_csv(io.BytesIO(uploaded['bluebikes_holdout.csv']))

# I'm then going to pre-process it using your commands.
holdout_predictors, holdout_labels = processData(bluebike_holdout)

# Then I'm going to evaluate your model's performance on that data.
loss_metrics = model.evaluate(holdout_predictors,holdout_labels,verbose=1)